In [2]:
# 1. Library
!pip install -U scikit-learn
!pip install nltk

import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# 2. Mempersiapkan Data

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Path ke folder dokumen
docs_path = '/content/drive/MyDrive/dataset/hadits' 

# Membaca dokumen
documents = []
doc_names = []
for filename in os.listdir(docs_path):
    if filename.endswith('.txt'):
        with open(os.path.join(docs_path, filename), 'r', encoding='utf-8') as file:
            documents.append(file.read())
            doc_names.append(filename)

print(f"Jumlah dokumen yang dimuat: {len(documents)}")

Jumlah dokumen yang dimuat: 40


In [5]:
# 3. Membuat Representasi Vektor dengan TF-IDF
# Gunakan stopwords dari NLTK
stop_words = stopwords.words('indonesian')

# Buat vectorizer dengan stopwords Indonesia
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=5000)

# Transformasi dokumen menjadi matriks TF-IDF
tfidf_matrix = vectorizer.fit_transform(documents)

# Cetak bentuk matriks TF-IDF
print("TF-IDF matrix shape:", tfidf_matrix.shape)

TF-IDF matrix shape: (40, 237)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


In [6]:
# 4. Implementasi Cosine Similarity untuk Pencarian
def search_quran(query, top_n=5):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_n:][::-1]
    results = []
    for idx in top_indices:
        results.append({
            'Dokumen': doc_names[idx],
            'Similarity Score': similarities[idx],
            'Isi Dokumen': documents[idx][:500] + '...' 
        })
    return results

In [7]:
# 5. Interface Sederhana untuk Question Answering

from IPython.display import display, HTML

def interactive_search():
    while True:
        query = input("Masukkan pertanyaan Anda (atau ketik 'exit' untuk keluar): ")
        if query.lower() == 'exit':
            break
        results = search_quran(query)
        for i, res in enumerate(results, 1):
            print(f"\nHasil {i}:")
            print(f"Dokumen: {res['Dokumen']}")
            print(f"Similarity Score: {res['Similarity Score']:.4f}")
            print(f"Isi Dokumen: {res['Isi Dokumen']}\n")

# Contoh 1
query = "Apa itu zakat?"
results = search_quran(query)
for i, res in enumerate(results, 1):
    print(f"\nHasil {i}:")
    print(f"Dokumen: {res['Dokumen']}")
    print(f"Similarity Score: {res['Similarity Score']:.4f}")
    print(f"Isi Dokumen: {res['Isi Dokumen']}\n")


Hasil 1:
Dokumen: hadits silaturahmi dengan orang tua dan saudara.txt
Similarity Score: 0.2974
Isi Dokumen: Beribadahlah pada Allah SWT dengan sempurna jangan syirik, dirikanlah sholat, tunaikan zakat, dan jalinlah silaturahmi dengan orangtua dan saudara....


Hasil 2:
Dokumen: hadits orang meninggal dunia.txt
Similarity Score: 0.0000
Isi Dokumen: Dari Abu Hurairah RA, Rasulullah bersabda, Jika ada orang yang meninggal dunia, maka malaikat berkata apa yang telah lalu amal, sedangkan manusia membicarakan apa yang ia tinggalkan warisan....


Hasil 3:
Dokumen: hadits adab meminta izin masuk ke rumah orang lain.txt
Similarity Score: 0.0000
Isi Dokumen: Suatu hari datang seorang dari Bani Amir ke rumah Rasulullah dan meminta izin untuk memasuki rumah beliau. Maka, Rasulullah berkata kepada pembantunya, 'Keluarlah kamu dan ajarkan laki-laki itu adab meminta izin, katakanlah padanya untuk mengucapkan, Assalamualaikum, bolehkah aku masuk....


Hasil 4:
Dokumen: hadits mengucapkan doa sebelum 

In [10]:
# Contoh 2
query = "mengucapkan salam"
results = search_quran(query)
for i, res in enumerate(results, 1):
    print(f"\nHasil {i}:")
    print(f"Dokumen: {res['Dokumen']}")
    print(f"Similarity Score: {res['Similarity Score']:.4f}")
    print(f"Isi Dokumen: {res['Isi Dokumen']}\n")


Hasil 1:
Dokumen: hadits keutamaan orang yang mengucap salam.txt
Similarity Score: 0.4443
Isi Dokumen: Sesungguhnya orang yang paling utama di sisi Allah adalah mereka yang memulai salam....


Hasil 2:
Dokumen: hadits keutamaan bertutur kata yang baik 2.txt
Similarity Score: 0.2667
Isi Dokumen: Dari Abu Syuraih, ia berkata pada Rasulullah, 'Wahai Rasulullah, tunjukkanlah padaku suatu amalan yang dapat memasukkanku ke dalam surga. Beliau bersabda, Di antara sebab mendapatkan ampunan Allah adalah menyebarkan salam dan bertutur kata yang baik....


Hasil 3:
Dokumen: hadits meminta izin masuk ke rumah orang lain.txt
Similarity Score: 0.0000
Isi Dokumen: Dari Abu Musa al-Asy'ary RA, ia berkata bahwa Rasulullah SAW bersabda, Mintalah izin tiga kali. Jika diizinkan maka itu hakmu, jika tidak maka kembali lah....


Hasil 4:
Dokumen: hadits mengucapkan doa sebelum masuk rumah.txt
Similarity Score: 0.0000
Isi Dokumen: Dengan menyebut Nama Allah kami masuk, dan dengan menyebut Nama Allah kami ke